In [ ]:
import os
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
from scipy import signal
from IPython.display import HTML, display

matplotlib.rcParams.update({'font.size':20, 'font.family':'DejaVu Sans'})

In [ ]:
# functions

def lambda_sq(center_freq: float, chan_width: float):
    """Equation 34
    Calculate the lambda squared value
    """
    lsq = (c**2 / center_freq**2) * \
        (1 + ( 0.75 * (chan_width/center_freq)**2))
    return lsq


def fpol(i, q, u):
    lpol = q + u*1j
    fpol = np.abs(lpol)/i
    return fpol


def lpol(q, u):
    lpol = q + u*1j
    return lpol


def mkdir(dir):
    if not os.path.isdir(dir):
        os.makedirs(dir)


mkpath = os.path.join

In [ ]:
mods = dict(
    test=dict(name="sim-data-test", los=3),
    normal=dict(name="sim-data", los=100)
    )


version = "normal"
odir_main, n_los = mods[version].values()



odir2 = mkpath(odir_main, "two", "los-data")


In [ ]:

# Delete previous
if os.path.isdir(odir_main):
    print("Deleting folder")
    os.system(f"rm -r {odir_main}/two")

In [ ]:
############################################################
# selected frequencies
sels = np.array([
    3, 4, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 42, 43,
    44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 60, 72, 73])

nans = np.array([_ for _ in range(80) if _ not in sels])

freqs = np.linspace(861_224_609.375, 1_706_461_914.0625, 80)
f0 = freqs[0]


mask_nf = np.zeros(len(freqs), dtype=bool)
mask_f = np.ones(len(freqs), dtype=bool)
mask_f[sels] = False


############################################################

spectral_index = -0.75

I0 = 4 #Jy.
I = I0 * pow((freqs/f0), spectral_index)

# generating some random noise centered around mean 0 and std 2mJy per beam
noise_std = 2e-3 #Jy/beam
noise_I = np.abs(np.random.normal(0, noise_std, I.size))

Iobserved = I + noise_I



C = 3e8
lam2 = (C/freqs)**2


## Two unresolved Components 

In [ ]:
mkdir(odir2+"-f")
mkdir(odir2+"-nf")
    
for j in range(n_los):
    # print(f"Creating data for region: ", j+1)
    noise_Q = np.abs(np.random.normal(0, 1e-3, len(freqs)))
    noise_U = np.abs(np.random.normal(0, 1e-3, len(freqs)))
    
       
    p0 = np.random.ranf()  # fractional p
    RM = np.random.randint(-150, 150) # rad/m^2
    PA = np.random.uniform(-np.pi/2.0, np.pi/2.0)
    
    
    # make sure this sigma RM is not too large because fitting will fail
    # i.e if prior is searching for RM btwn 150 and -150, and this val
    # hs a max of 200, then the max value will be (350), and you will 
    # get problems
    sig = np.random.uniform(0, 40)
    sig2 = np.random.uniform(0, 20)


    # ensure that this is smaller than p0
    p02 = p0 * 0.5  # fractional p
    # make this smaller so that the peak is more prominent
    # RM2 = RM + np.random.randint(-40, 0) # rad/m^2
    RM2 = np.random.randint(-150, 150)
    PA2 = np.random.uniform(-np.pi/2.0, np.pi/2.0)
    

    p = p0 *  np.exp(2j * PA) * np.exp(2j * RM * lam2) \
        + p02 *  np.exp(2j * PA2) * np.exp(2j * RM2 * lam2)
    

    # orig model
    # p = (p0 *  np.exp(2j * PA) * np.exp(2j * RM * lam2) \
    #     + p02 *  np.exp(2j * PA2) * np.exp(2j * RM2 * lam2)) / 2.0#* (1.0/np.sqrt(2))

    q = p.real
    u = p.imag
    
    Q = q * Iobserved + noise_Q
    U = u * Iobserved + noise_U
 
 
    #sigma_Q = q * ( (noise_q/q)**2 + (noise_I/Iobserved)**2 )**0.5 
    #sigma_U = u * ( (noise_q/u)**2 + (noise_I/Iobserved)**2 )**0.5 

    #Q = Q + noise_Q
    #U = U + noise_U

    data = dict()
    data["freqs"] = freqs
    data["lambda_sq"] = lam2
    data["chan_width"] = freqs
    data["fpol"] = fpol(Iobserved, Q, U)
    data["lpol"] = lpol(Q, U)


    data["fpol_err"] = np.zeros(freqs.size)
    data["lpol_err"] = np.zeros(freqs.size)
    data["noise"] = np.zeros(freqs.size)
    data["pangle_err"] = np.zeros(freqs.size)


    data["pangle"] = np.zeros(freqs.size)
    data["snr"] = np.zeros(freqs.size)
    data["psnr"] = np.zeros(freqs.size)
    data["mask"] = np.zeros(freqs.size)

    data["I"] = Iobserved
    data["Q"] = Q
    data["U"] = U
    data["Q_err"] = noise_Q
    data["U_err"] = noise_U
    data["I_err"] = noise_I

    # Make my data look alike
    data_nf = {k: np.ma.masked_array(data=v, mask=mask_nf).compressed() for k, v in data.items()}

    data_f = {k: np.ma.masked_array(data=v, mask=mask_f).compressed() for k, v in data.items()}

    data_nf["in_rm"] = RM
    data_nf["in_pa"] = PA
    data_nf["in_p0"] = p0
    data_nf["in_sigma_rm"] = sig
    data_nf["in_rm2"] = RM2
    data_nf["in_pa2"] = PA2
    data_nf["in_p02"] = p02

    data_f["in_rm"] = RM
    data_f["in_pa"] = PA
    data_f["in_p0"] = p0
    data_f["in_sigma_rm"] = sig
    data_f["in_rm2"] = RM2
    data_f["in_pa2"] = PA2
    data_f["in_p02"] = p02
    
    np.savez(mkpath(odir2+"-nf", f"reg_{j+1}.npz"), **data_nf)
    np.savez(mkpath(odir2+"-f", f"reg_{j+1}.npz"), **data_f)

print("----------")
print("Data read")

# input_param = np.vstack((input_p0, input_PA, input_RM, input_dRM))
# np.savetxt('MOCK/DDP/INPUT-PARAMETERS.txt', input_param.T)


In [ ]:
# run all cells above alone
print("run all cells above alone")

# ln -s sim-data/two/los-data-f weirdo-data
# ln -s sim-data/two/los-rm-data-f weirdo-rm-data
# python qu_pol/scrappy/rmsynthesis/rm_synthesis.py -id sim-data/two/los-data-f -od sim-data/two/los-rm-data-f -md 400 --depth-step 1 -np
# python plt-script.py


In [ ]:
# q = Q/Iobserved
# u = U/Iobserved
# P = (Q**2 + U**2)**0.5
# q_noise = abs(q * ( (noise_Q/Q)**2 + (noise_I/I)**2 )**0.5)
# u_noise = abs(u * ( (noise_U/U)**2 + (noise_I/I)**2 )**0.5)

# # aka polarised noise
# p_noise = ( (Q/P)**2 * noise_Q**2 + (U/P)**2 * noise_U**2 )**0.5
# sigma_p = abs(p) * ((p_noise/P)**2 + (noise_I/I)**2 )**0.5
# sigma_p.max()


# # plot total intensity without noise, noise distibution and TI with noise

# plt.close("all")
# fig, ax = plt.subplots(figsize=(16,9), ncols=3, nrows=2, sharex=False, sharey=False)
# ax[0,0].errorbar(freqs/1e9, I, yerr=noise_I, fmt='bo', ecolor='r', ms=2)
# # ax[0,0].set_xscale('log')
# # ax[0,0].set_yscale('log')
# ax[0,0].set_ylabel('Total Intensity [Jy pixel$^{-1}$]')
# ax[0,0].set_xlabel('Frequency [Hz]')
# # ax[0,0].savefig('MOCK/EXAMPLE-I-MODEL.pdf')



# ax[0,1].hist(noise_I, bins=100, color='b', alpha=0.5)
# ax[0,1].set_xlabel('noise in I [Jy pixel$^{-1}$]')
# ax[0,1].set_ylabel('Count')
# # ax[0,1].savefig('MOCK/EXAMPLE-I-DISTRIBUTION.pdf')


# ax[0,2].plot(freqs, Iobserved, 'bo', ms=2)
# # ax[0,2].set_xscale('log')
# # ax[0,2].set_yscale('log')
# ax[0,2].set_ylabel('Total Intensity [Jy beam$^{-1}$]')
# ax[0,2].set_xlabel('Frequency [Hz]')



# ax[1,0].errorbar(lam2, q, yerr=q_noise, fmt='ro', ecolor='y', ms=2, label='q')
# ax[1,0].errorbar(lam2, u, yerr=u_noise, fmt=1,'bo', ecolor='y', ms=2, label='u')
# ax[1,0].xlabel('Wavelength [m$^{2}$]')
# ax[1,0].ylabel('Fractional Polarisation')
# ax[1,0].legend(loc='best')

# ax[1,1].hist(noise_Q, bins=100, color='c', label='$N_Q$')
# ax[1,1].hist(noise_U, bins=100, color='pink', label='$N_U$')
# ax[1,1].xlabel('noise [Jy pixel$^{-1}$]')
# ax[1,1].ylabel('Count')
# ax[1,1].tight_layout()
# ax[1,1].legend(loc='best')

# # fig.tight_layout()
# # fig.show()

### Run for Debugggg

In [ ]:
# some other pltos

loss = range(1, 101)
fm, ufm, fm2, ufm2, af, auf, af2, auf2 = [], [], [], [], [], [], [], []
for i in range(1, n_los+1):

    flagged_model = f"{odir_main}/two/qu-fits-f-single/qufit-reg_{i}-fitparameters.txt"
    unflagged_model = f"{odir_main}/two/qu-fits-nf-single/qufit-reg_{i}-fitparameters.txt"

    actual_f = f"sim-data/two/los-data-f/reg_{i}.npz" 
    actual_nf = f"sim-data/two/los-data-nf/reg_{i}.npz"

    fm.append(np.loadtxt(flagged_model)[2])
    ufm.append(np.loadtxt(unflagged_model)[2])

    fm2.append(np.loadtxt(flagged_model)[6])
    ufm2.append(np.loadtxt(unflagged_model)[6])

    af.append(np.load(actual_f)["in_rm"])
    auf.append(np.load(actual_nf)["in_rm"])

    af2.append(np.load(actual_f)["in_rm2"])
    auf2.append(np.load(actual_nf)["in_rm2"])

fm = np.array(fm)
ufm = np.array(ufm)
fm2 = np.array(fm2)
ufm2 = np.array(ufm2)

af = np.array(af)
auf = np.array(auf)
af2 = np.array(af2)
auf2 = np.array(auf2)


fig, ax = plt.subplots(ncols=2, figsize=(16,6))
# ax[0].plot(loss, fm, "o", label="flagged mod", linestyle="--")
# ax[0].plot(loss, af, "o", label="actual flagged")
# ax[0].plot(loss, af-fm, "o", label="Actual - model [with flagging]")
ax[0].axhline(0, color="k")
ax[0].legend()

# ax[1].plot(loss, ufm, "o", label="unflagged mod", linestyle="--")
# ax[1].plot(loss, auf, "o", label="actual unflagged")
# ax[1].plot(loss, auf-ufm, "o", label="Actual - model [No flagging]")
ax[1].axhline(0, color="k")
ax[1].legend()
